# Машинное обучение

In [306]:
# подгрузим необходимые библиотеки для проведения вычислений и построения моделей
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import HuberRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso, Ridge

import warnings
warnings.filterwarnings("ignore")

В заключительной части проекта нами будет выполнено построение различных моделей машинного обучения с целью корректного прогнозирования стоимости подержанного автомобиля по его характеристикам. За основную модель была выбрана линейная регрессия, которая была усовершенствована с помощью применения маштабирования признаков и последующего использования регуляризаций и подбора их гиперпараметров. В качестве оценки корректности работы модели мы использовали среднеквадратичную ошибку и поставили перед собой задачу ее минимизации. 

In [307]:
cars_df = pd.read_csv('cars_df_final.csv')
print(cars_df.shape)
cars_df.head()

(2542, 23)


,Brand,Year,Exterior color,Interior color,Drivetrain,Fuel type,Capacity,Configuration,Valves,Accidents or damage,...,MPG_max,MPG_mean,Mileage,Usage Intensity,Ownership History,Engine Power,Fuel Efficiency,Origin,Gallons Per Year,Price
0,Hyundai,2022,Red,Black,Front-wheel Drive,Gasoline,2.5,I4,16.0,At least 1 accident or damage reported,...,28.000000,25.000000,13256.0,6628.000,Potential Risk,40.0,10.000000,Asian,265.120000,38988.0
1,Lexus,2016,Black,Black,All-wheel Drive,Gasoline,3.5,V6,24.0,At least 1 accident or damage reported,...,26.000000,22.500000,100067.0,12508.375,Bad,84.0,6.428571,Asian,555.927778,19747.0
2,Chevrolet,2017,White,Black,Front-wheel Drive,Hybrid,1.5,I4,16.0,At least 1 accident or damage reported,...,27.239548,23.916163,39032.0,5576.000,Potential Risk,24.0,15.944109,American,233.147769,25999.0
3,Mercedes-Benz,2022,White,Other,All-wheel Drive,Gasoline,3.0,I6,24.0,None reported,...,27.239548,23.916163,10901.0,5450.500,Good,72.0,7.972054,European,227.900272,96750.0
4,Land Rover,2020,Blue,Other,Four-wheel Drive,Gasoline,3.0,I6,24.0,None reported,...,22.000000,19.500000,24946.0,6236.500,Good,72.0,6.500000,British,319.820513,61530.0


In [308]:
# Разобъем данные на обучающую и тестовую выборку в пропорции 7:3
y = cars_df["Price"]
X = cars_df.drop(columns=["Price"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=77)

## 1. Гипотезы

Помимо обучения моделей в данной части нашего проекта мы так же рассмотрим несколько гипотез, которые построены непосредственно на м.о. и проведем их тестирование на основе полученных результатов.

* *Пробег автомобиля отрицательно влияет на цену*: качество автомобиля с ростом пробега снижается


* *Год выпуска положительно влияет на цену*: при прочих равных чем новее автомобиль, тем он дороже


* *Происхождение автомобиля "British" положительно влияет на цену*: среди британских автомобилей в основном представлены только премиальные бренды, поэтому, на наш взгляд, вес этого признака будет большим


* *Признак "Ownership History" будет значительно влиять на цену: плохая история автомобиля будет отрицательно влиять на цену, а хорошая - положительно

## 2. Маштабирование и кодирование признаков

При построении моделей машинного обучения крайне важным оказывается приведение признаков к единому маштабу с целью исключения моментов переобучения и неверной интерпретации весов отдельных признаков. Более того, необходимым оказывается кодирование категориальных признаков с помощью One-hot-Encoding, без применения которого обучение линейной регресии не предоставляется возможным.

In [309]:
# Разделим признаки по значениям
numeric_features = ['Year', 'Capacity','Valves', 'MPG_min', 'MPG_max', 'MPG_mean', 'Mileage',
                    'Usage Intensity', 'Engine Power', 'Fuel Efficiency', 'Gallons Per Year']


categorical_features = ['Brand', 'Exterior color', 'Interior color', 'Drivetrain', 'Fuel type', 'Configuration', 
                        'Accidents or damage', '1-owner vehicle', 'Personal use only', 'Ownership History', 'Origin']

In [312]:
column_transformer = ColumnTransformer([
    ('scaling', StandardScaler(), numeric_features),    
    ('ohe', OneHotEncoder(handle_unknown="ignore", drop="first"), categorical_features)
])

In [316]:
X_train_scaled = pd.DataFrame(column_transformer.fit_transform(X_train).toarray())
X_test_scaled = pd.DataFrame(column_transformer.transform(X_test).toarray())

In [319]:
# Получим названия признаков в новом DataFrame
feature_names = column_transformer.get_feature_names_out()
feature_names

array(['scaling__Year', 'scaling__Capacity', 'scaling__Valves',
       'scaling__MPG_min', 'scaling__MPG_max', 'scaling__MPG_mean',
       'scaling__Mileage', 'scaling__Usage Intensity',
       'scaling__Engine Power', 'scaling__Fuel Efficiency',
       'scaling__Gallons Per Year', 'ohe__Brand_Alfa Romeo',
       'ohe__Brand_Aston Martin', 'ohe__Brand_Audi', 'ohe__Brand_BMW',
       'ohe__Brand_Bentley', 'ohe__Brand_Buick', 'ohe__Brand_Cadillac',
       'ohe__Brand_Chevrolet', 'ohe__Brand_Chrysler', 'ohe__Brand_Dodge',
       'ohe__Brand_FIAT', 'ohe__Brand_Ferrari', 'ohe__Brand_Ford',
       'ohe__Brand_GMC', 'ohe__Brand_Genesis', 'ohe__Brand_Honda',
       'ohe__Brand_Hummer', 'ohe__Brand_Hyundai', 'ohe__Brand_INFINITI',
       'ohe__Brand_Jaguar', 'ohe__Brand_Jeep', 'ohe__Brand_Kia',
       'ohe__Brand_Lamborghini', 'ohe__Brand_Land Rover',
       'ohe__Brand_Lexus', 'ohe__Brand_Lincoln', 'ohe__Brand_Lotus',
       'ohe__Brand_MINI', 'ohe__Brand_Maserati', 'ohe__Brand_Mazda',
       

## 3. Обучение Линейной Регрессии

Для начала обучим модель линейной регрессии, которая идеально подходит под выбранные нами данные. В таком случае мы будет подбирать оптимальные веса признаков с целью дальнейшей минимизации ошибки на тесте.

### 3.1. Базовая модель

In [465]:
lr_base = LinearRegression()  # обучаем модель линейной регрессии 
lr_base.fit(X_train_scaled, y_train)

y_train_pred = lr_base.predict(X_train_scaled)
y_pred = lr_base.predict(X_test_scaled)

После обучения посмотрим на получившиеся значения ошибок:

In [466]:
print(f"Train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred)}")
print()
print(f"Train MAE = {mean_absolute_error(y_train, y_train_pred)}")
print(f"Test MAE = {mean_absolute_error(y_test, y_pred)}")

Train MSE = 386320960.93131673
Test MSE = 16099721281.448334

Train MAE = 9636.108206857785
Test MAE = 15001.226572739188


In [467]:
# Создадим таблицу с весами
lr_base_feature_coefs = pd.DataFrame(
    data=lr_base.coef_, index=feature_names, columns=['Coef']
)

lr_base_feature_coefs

,Coef
scaling__Year,1.099119e+04
scaling__Capacity,2.020812e+04
scaling__Valves,1.415034e+04
scaling__MPG_min,1.047086e+16
scaling__MPG_max,1.000383e+16
...,...
ohe__Ownership History_Good,-4.919259e+15
ohe__Ownership History_Potential Risk,-2.459630e+15
ohe__Origin_Asian,7.637151e+14
ohe__Origin_British,9.786240e+14


__Вывод:__ Итак, посмотрев на получившиеся оптимальные веса модели, нетрудно заметить симптом переобучения, подгонки нашей модели под данные, - большие веса у признаков. Более того, обобщающая способность модели получается низкой, ведь она оказывается не способна выдавать хорошие резльтаты как на тренировочной выборке, так и на тестовой.

### 3.2. HuberRegressor - попытка снизить влияние выбросов

Одной из причин переобучения нашей модели может быть большое количество выбросо, под которые она стремиться подстроиться, минимизируя среднюю ошибку, тем самым увеличивая ее на нормальных объектах. Для борьбы с выбросами можно использовать функцию потерь Huber, которая вблизи нуля ведет себя, как MSE, а при отдалении от нулевого значения ошибки представляет из себя MAE. Подобная функция оказывается робастной, то есть устойчивой к возникновению выбросов в модели.

In [359]:
eps = np.linspace(1, 10, 5)
alphas = np.linspace(1e-5, 5, 30)

In [367]:
# подберем оптимальные гиперпараметры
lr_robust_GS = GridSearchCV(  
    estimator=HuberRegressor(),
    param_grid={'epsilon': eps, 
                'alpha': alphas}
)

lr_robust_GS.fit(X_train_scaled, y_train);

In [371]:
best_alpha, best_epsilon = lr_robust_GS.best_params_.values()
best_alpha, best_epsilon

(1e-05, 3.25)

In [372]:
# обучим лучшую из моделей 
lr_robust = HuberRegressor(alpha=best_alpha, epsilon=best_epsilon)
lr_robust.fit(X_train_scaled, y_train)

HuberRegressor(alpha=1e-05, epsilon=3.25)

In [373]:
y_train_pred = lr_robust.predict(X_train_scaled)
y_pred = lr_robust.predict(X_test_scaled)

Теперь давайте посмотрим на значения ошибок на тесте и на трейне:

In [374]:
print(f"Train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred)}")
print()
print(f"Train MAE = {mean_absolute_error(y_train, y_train_pred)}")
print(f"Test MAE = {mean_absolute_error(y_test, y_pred)}")

Train MSE = 479082988.72271305
Test MSE = 17428551484.513905

Train MAE = 9028.792039611633
Test MAE = 14499.46284012911


In [375]:
# Создадим таблицу с весами
lr_robust_feature_coefs = pd.DataFrame(
    data=lr_robust.coef_, index=feature_names, columns=['Coef']
)

lr_robust_feature_coefs

,Coef
scaling__Year,10191.343828
scaling__Capacity,22731.277231
scaling__Valves,18925.835079
scaling__MPG_min,936.648510
scaling__MPG_max,-3053.940734
...,...
ohe__Ownership History_Good,531.508539
ohe__Ownership History_Potential Risk,397.380238
ohe__Origin_Asian,-2327.823545
ohe__Origin_British,54624.683454


__Вывод:__ Таким образом, мы смогли добиться лишь незначительного снижения средних ошибок, а рассмотренные веса по-прежнему остаются достаточно большими. Следовательно, ограничения влияния выбросов не дает значительных результатов по минимизации нашей функции потерь, возможно, выбросов оказывается и не так уж и много.

### 3.3. Lasso-регрессия

Воспользуемся теперь другим способом регуляризации, который представляет из себя добавление суммы модулей весов к выбранной функции потерь с определенным параметром $\alpha$. Особенность Lasso-регрессии заключается в обнулении некоторых весов, что мы и увидим при дальнейшем обучении модели.

In [475]:
# подбираем оптимальный гиперпараметр альфа
alphas = np.linspace(0.5, 20, 40)

lasso_GS = GridSearchCV(
    Lasso(),
    param_grid={'alpha': alphas}
)

lasso_GS.fit(X_train_scaled, y_train);

In [476]:
best_alpha = lasso_GS.best_params_['alpha']
best_alpha

20.0

In [477]:
# обучаем лучшую модель
lasso_lr = Lasso(alpha=best_alpha)
lasso_lr.fit(X_train_scaled, y_train)

Lasso(alpha=20.0)

In [478]:
y_train_pred = lasso_lr.predict(X_train_scaled)
y_pred = lasso_lr.predict(X_test_scaled)

Вновь посмотрим на значения аших ошибок на тестовой и обучающей выборках:

In [479]:
print(f"Train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred)}")
print()
print(f"Train MAE = {mean_absolute_error(y_train, y_train_pred)}")
print(f"Test MAE = {mean_absolute_error(y_test, y_pred)}")

Train MSE = 393034426.64557415
Test MSE = 16127064299.403112

Train MAE = 9694.258907858393
Test MAE = 14954.359618814167


In [480]:
# Создадим таблицу с весами
lasso_lr_feature_coefs = pd.DataFrame(
    data=lasso_lr.coef_, index=feature_names, columns=['Coef']
)

lasso_lr_feature_coefs

,Coef
scaling__Year,10798.920597
scaling__Capacity,9034.026172
scaling__Valves,5942.509582
scaling__MPG_min,-0.000000
scaling__MPG_max,-2619.793359
...,...
ohe__Ownership History_Good,-0.000000
ohe__Ownership History_Potential Risk,1247.861045
ohe__Origin_Asian,-1954.481466
ohe__Origin_British,36950.211755


__Вывод:__ Нетрудно заметить, что вновь ощутимого улучшения качества модели не наблюдается, но при этом веса удалось снизить, а некоторые даже занулить, о чем мы и говорили ранее.

### 3.4. Ridge-регрессия

Посмотрим теперь еще на один вид регуляризации - Ridge, который представляет из себя добавление суммы квадратов нормы весов.

In [398]:
# вновь подберем оптимальные гиперпараметры модели
solvers = ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']

solver_GS = GridSearchCV(
    Ridge(),
    param_grid={'solver': solvers}
)

solver_GS.fit(X_train_scaled, y_train)
best_solver = solver_GS.best_params_['solver']
best_solver

'saga'

In [399]:
alphas = np.linspace(0.5, 20, 40)

ridge_GS = GridSearchCV(
    Ridge(solver=best_solver),
    param_grid={'alpha': alphas}
)

ridge_GS.fit(X_train_scaled, y_train);

In [400]:
best_alpha = ridge_GS.best_params_['alpha']
best_alpha

3.0

In [401]:
# обучаем лучшую из моделей
ridge_lr = Ridge(solver=best_solver, alpha=best_alpha)
ridge_lr.fit(X_train_scaled, y_train)

Ridge(alpha=3.0, solver='saga')

In [403]:
y_train_pred = ridge_lr.predict(X_train_scaled)
y_pred = ridge_lr.predict(X_test_scaled)

Посмотрим на значения ошибок:

In [404]:
print(f"Train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred)}")
print()
print(f"Train MAE = {mean_absolute_error(y_train, y_train_pred)}")
print(f"Test MAE = {mean_absolute_error(y_test, y_pred)}")

Train MSE = 458230330.8987806
Test MSE = 16693218996.43311

Train MAE = 10175.360674660777
Test MAE = 15458.919499120584


In [405]:
# Создадим таблицу с весами
ridge_lr_feature_coefs = pd.DataFrame(
    data=ridge_lr.coef_, index=feature_names, columns=['Coef']
)

ridge_lr_feature_coefs

,Coef
scaling__Year,10899.832818
scaling__Capacity,10820.912791
scaling__Valves,9112.976509
scaling__MPG_min,865.703845
scaling__MPG_max,-2774.319153
...,...
ohe__Ownership History_Good,-52.788879
ohe__Ownership History_Potential Risk,1737.936166
ohe__Origin_Asian,-6807.163366
ohe__Origin_British,39261.136485


__Вывод:__ Аналогично Lasso-регрессии радикального улучшения качества модели не случилось, однако мы вновь смогли добиться снижения весов из-за наложения ограничения на большие веса.  

### 3.5. Стохастический градиентный спуск

Давайте теперь попробуем обучить нашу линейную регрессию с помощью стохастического градиентного спуска. Градиентный спуск - это оптимизационный алгоритм, который находит минимум функции, двигаясь в направлении, обратном градиенту, т.е. в направлении наискорейшего убывани фуекции. Он основывается на вычислении градиента функции в текущей точке и обновлении переменных с определенной скоростью обучения. Стохастический же градиентый спуск на каждой из итераций будет считать антиградиент на случайном подмножестве объектов, тем самым обеспечивая более хаотичную траекторию движения и возможность перескочить через локальный минимум в поисках глобального.

In [413]:
# подбор гиперпараметров модели
av_loss = ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']

loss_GS = GridSearchCV(
    estimator=SGDRegressor(),
    param_grid={'loss': av_loss}
)

best_loss = loss_GS.fit(X_train_scaled, y_train).best_params_['loss']
best_loss

'squared_epsilon_insensitive'

In [414]:
alphas = np.linspace(0.5, 20, 40)
penalties = ['l2', 'l1', 'elasticnet']

sgd_GS = GridSearchCV(
    estimator=SGDRegressor(loss=best_loss),
    param_grid={
        'penalty': penalties,
        'alpha': alphas
    }
)

sgd_GS.fit(X_train_scaled, y_train);

In [415]:
best_alpha, best_penalty = sgd_GS.best_params_.values()
best_alpha, best_penalty

(1.5, 'l1')

In [416]:
# обучаем лучшую из моделей
sgd_lr = SGDRegressor(loss=best_loss, alpha=best_alpha, penalty=best_penalty)
sgd_lr.fit(X_train_scaled, y_train)

SGDRegressor(alpha=1.5, loss='squared_epsilon_insensitive', penalty='l1')

In [417]:
y_train_pred = sgd_lr.predict(X_train_scaled)
y_pred = sgd_lr.predict(X_test_scaled)

Теперь посмотрим на получившиеся у нас ошибки

In [419]:
print(f"Train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred)}")
print()
print(f"Train MAE = {mean_absolute_error(y_train, y_train_pred)}")
print(f"Test MAE = {mean_absolute_error(y_test, y_pred)}")

Train MSE = 498235494.36949384
Test MSE = 16843216647.081367

Train MAE = 10432.18644263375
Test MAE = 15839.776643234038


In [420]:
# Создадим таблицу с весами
sgd_lr_feature_coefs = pd.DataFrame(
    data=sgd_lr.coef_, index=feature_names, columns=['Coef']
)

sgd_lr_feature_coefs

,Coef
scaling__Year,10473.199210
scaling__Capacity,8140.417827
scaling__Valves,5696.787842
scaling__MPG_min,205.328840
scaling__MPG_max,-3185.650027
...,...
ohe__Ownership History_Good,0.000000
ohe__Ownership History_Potential Risk,2002.986151
ohe__Origin_Asian,-3873.953594
ohe__Origin_British,35368.813301


__Вывод:__ Однако применение стохастический градиентного спуска также ничего радикально не изменил, и нага ошибка на тесте и на трейне осталась примерно на том же уровне. Более того, мы вновб наблюдаем большие значения у весов, что может сигнализировать о подгонке модели под данные.

## 4. Алгоритм имитация отжига

Помимо градиентного спуска в машинном обучении существует "Алгоритм имитации отжига", который являестя оптимизацией среднеквадратичной ошибки для линейной регрессии. По сути алгоритм иммитирует процесс отжига, постепенно исследуя окрестности текущего состояния. На каждой итерации генерируется новое состояние с измененными параметрами модели. Если новое состояние улучшает MSE, оно принимается, иначе может быть принято с вероятностью, уменьшающейся со временем. Алгоритм продолжается до достижения условия остановки. 

Давайте пропишем класс, который будет осуществлять описанный выше алгоритм

In [278]:
class otzhig:
    def __init__(self, temp_init=1, temp_fin=0.01, n=150): 
        self.temp_init = temp_init
        self.temp_fin = temp_fin
        self.n = n
        self.lr = LinearRegression()


    def vesa(self, old_w):  # запишем функцию создающую новые веса по формуле, как указано выше
        new_w = old_w + np.random.standard_t(3, size=old_w.shape)
        return new_w


    def fit(self, X, y):  # обучим модель
        w_init = np.random.normal(0,1,X.shape[1])  # как раньше сгенерируем веса из нормального распределения по размеру X.shape[1]
        self.lr.fit(X, y)  # обучаем модель
        loss_cur = mean_squared_error(y, self.lr.predict(X))  # смотрим на изначальную ошибку
        
        for i in range(self.n):  # запишем цикл, производящий сравнение 
            new_w = self.vesa(w_init)
            self.lr.fit(X, y)
            loss_n = mean_squared_error(y, self.lr.predict(X))  # смотрим на новую ошибку
            
            if loss_n < loss_cur:  # сравниваем и перезаписываем меньшую из них, если меньше
                w_init = new_w
                loss_cur = loss_n
                
            else:  # иначе совершаем перерасчет вероятности принятия новых весов и параметром температуры 
                temp =  self.temp_init * ((self.temp_fin / self.temp_init)**(i / self.n))
                prob = np.exp(-(loss_n - loss_cur) / temp)
                a = np.random.rand()
                if a < prob:
                    w_init = new_w
                    loss_cur = loss_n
    
        return w_init, loss_cur

Теперь обучим нашу модель и посмотрим на оптимальные веса для каждого из признаков.

In [283]:
model = otzhig().fit(X_train_scaled, y_train);

In [286]:
print("Оптимальные веса для регрессии:")
print(model[0])
print()
print("Конечное значение ошибки на трейне:", model[1])

Оптимальные веса для регрессии:
[ 21.29026116  -8.61205467 -17.32532386  16.63599264 -31.12616941
 -12.37757961 -31.5280681  -15.31394233 -38.26088759  26.73672034
  42.31934677   7.26316338  45.0872935   24.4815187   41.47668416
 -13.86721388 -11.7859219   -3.20657292 -32.47229054  12.39828571
   9.42691047   7.81910566  -6.38658949  23.3139619   16.85213268
  47.48247239  35.96354677  -9.68756819  32.94031053  24.14032805
  -1.41390455 -32.14677201  21.2191973    1.29911148  -2.34219405
   7.16723394   5.70188872 -35.22820059  -4.53363441 -10.79591494
 -21.69301397 -16.44352388 -30.11216149  28.57459681  10.24882141
  52.32485443 -22.11830071 -15.37152008 -38.13079245 -11.99172075
  -8.260969    37.79325521  -8.26541981 -14.12858984 -19.20040145
 -13.438808     1.71326251 -23.44331605   2.00624188  -7.75911834
 -27.01393925  39.33782503  -1.91436004 -13.45868147 -35.76491008
 -22.12756484 -20.75120423  -9.73489779  27.30683245   5.10395737
 -36.5333749  -12.60411319  11.99109145   4.

In [287]:
model = otzhig().fit(X_test_scaled, y_test)
print("Конечное значение ошибки на тесте:", model[1])

Конечное значение ошибки на тесте: 6157385318.926605


__Вывод:__ Таким образом, нам удалось уменьшить значение нашей ошибки на тесте по сравнению с обычной линейной регрессией, причем на порядок. 

## 5. Случайный лес

Теперь обучим нашу модель с применение случайного леса. В машинном обучении случайный лес считается довольно распространенный алгоритмом, который помогает улучшать качество построенных моделей за счет независимого обучения большого числа деревьев. Более того, данный алгоритм не склонен к переобучению, ведь даже при увеличении числа деревьев ошибка выходит на асимптоту.

Применими модель Random Forest и произведем перебор по сетке с целью нахождения оптимальный гиперпараметров.

In [421]:
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300],
    'max_depth': [3, 5, 10, 15],
    'min_samples_split': [2, 5, 10]
}

rf_GS = GridSearchCV(
    RandomForestRegressor(), param_grid, cv=4, scoring='neg_mean_squared_error'
)

rf_GS.fit(X_train_scaled, y_train);

In [425]:
rf_GS.best_params_

{'max_depth': 15, 'min_samples_split': 2, 'n_estimators': 100}

In [423]:
# Наилучшая максимальная глубина - 15, что подтверждает идею о том, что в ансамблях базовые модели должны иметь как можно меньшее смещение
best_max_depth, best_min_samples_split, best_n_estimators = rf_GS.best_params_.values()
best_max_depth, best_min_samples_split, best_n_estimators

(15, 2, 100)

In [441]:
rf_model = RandomForestRegressor(
    n_estimators=best_n_estimators, min_samples_split=best_min_samples_split, max_depth=best_max_depth
)

rf_model.fit(X_train_scaled, y_train);

In [442]:
y_train_pred = rf_model.predict(X_train_scaled)
y_pred = rf_model.predict(X_test_scaled)

In [443]:
print(f"Train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred)}")
print()
print(f"Train MAE = {mean_absolute_error(y_train, y_train_pred)}")
print(f"Test MAE = {mean_absolute_error(y_test, y_pred)}")

Train MSE = 69235373.72802378
Test MSE = 15514818419.882542

Train MAE = 3180.3070552671143
Test MAE = 11335.995306198025


In [446]:
# Создадим таблицу с важностями
rf_model_feature_coefs = pd.DataFrame(
    data=rf_model.feature_importances_, index=feature_names, columns=['Coef']
)

rf_model_feature_coefs

,Coef
scaling__Year,0.064385
scaling__Capacity,0.020803
scaling__Valves,0.209913
scaling__MPG_min,0.009701
scaling__MPG_max,0.018115
...,...
ohe__Ownership History_Good,0.001629
ohe__Ownership History_Potential Risk,0.007820
ohe__Origin_Asian,0.002803
ohe__Origin_British,0.009284


__Вывод:__ Таким образом, нам удалось добиться улучшения модели, и наша ошибка на тестовой и обучающей выборке стала значительно меньше, чем при использовании линейной регресии. Однако таких же хороший результатов как при использовании имитация отжига добиться не удалось. Но при этом мы решили проблему переобучения, снизив значения оптимального набора весов. 

## 6. Градиентный бустинг

В заключении подбора оптимального алгоритма машинного обучения для предсказания стоимости подержанного автомобиля на основании его характеристик мы воспользуемся градиентный бустингом. В современном машинного обучении градиентный бустинг является одной из самых передовых моделей, который представляет из себя композицию из n базовых моделей, каждая из которых строится последовательно с целью исправления ошибки предыдущей. Базовая идея градиентного бустинга - попытка минимизировать ошибку уже имеющейся модели при добавлении последующей.

In [447]:
# Подбираем оптимальные гиперпараметры модели
btg_raw_model_GS = GridSearchCV(
    estimator=GradientBoostingRegressor(),
    param_grid={'n_estimators': [50, 100, 150, 200]}
)

btg_raw_model_GS.fit(X_train_scaled, y_train)

# Определим оптимальное количество базовых моделей
best_n_estimators = btg_raw_model_GS.best_params_['n_estimators']
best_n_estimators

200

In [448]:
# Подбираем оптимальные гиперпараметры модели
btg_model_GS = GridSearchCV(
    estimator=GradientBoostingRegressor(n_estimators=best_n_estimators),
    param_grid={
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 5, 10, 15],
        'max_depth': [3, 5, 10, 15]
    }
)

btg_model_GS.fit(X_train_scaled, y_train);

In [449]:
btg_model_GS.best_params_

{'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5}

In [451]:
best_max_depth, best_min_samples_leaf, best_min_samples_split = btg_model_GS.best_params_.values()
best_max_depth, best_min_samples_leaf, best_min_samples_split

(5, 1, 5)

In [452]:
# Обучаем лучшую из моделей
btg_model = GradientBoostingRegressor(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    min_samples_leaf=best_min_samples_leaf,
    min_samples_split=best_min_samples_split
)

btg_model.fit(X_train_scaled, y_train)

GradientBoostingRegressor(max_depth=5, min_samples_split=5, n_estimators=200)

In [453]:
y_train_pred = btg_model.predict(X_train_scaled)
y_pred = btg_model.predict(X_test_scaled)

Посмотрим на получившиеся у нас значения ошибок на тесте и на трейне:

In [454]:
print(f"Train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred)}")
print()
print(f"Train MAE = {mean_absolute_error(y_train, y_train_pred)}")
print(f"Test MAE = {mean_absolute_error(y_test, y_pred)}")

Train MSE = 14211769.53381083
Test MSE = 14338503680.141674

Train MAE = 2773.4334510464682
Test MAE = 11065.955218999474


In [455]:
# Создадим таблицу с важностью признаков в модели градиентного бустинга
btg_model_feature_importances = pd.DataFrame(
    data=btg_model.feature_importances_, index=feature_names, columns=['Feature Importance']
)

btg_model_feature_importances

,Feature Importance
scaling__Year,0.085576
scaling__Capacity,0.016492
scaling__Valves,0.239467
scaling__MPG_min,0.005972
scaling__MPG_max,0.009955
...,...
ohe__Ownership History_Good,0.000199
ohe__Ownership History_Potential Risk,0.004379
ohe__Origin_Asian,0.003584
ohe__Origin_British,0.021569


__Вывод:__ Таким образом, применение градиентого бустинга помогло нам добиться значительного улучшения работы нашей модели, сильно снизив величины средних ошибок на обучении и на тесте, тем самым повысив обобщающую способность алгоритма.

## 7. Финальный вывод

__Вывод:__ Таким образом, среди примененных нами стандартных моделей машинного обучения наилучшие результаты показал градиентный бустинг, который оказался способен обеспечить минимальные зачения функции потерь как и на обучающей выборке, так и на тестовой. 

Именно поэтому при рассмотрении дальнейших гипотез о важности признаков мы будет пользоваться результатами обучения градиентного бустинга. Для гипотез же связанных с направлением влияния различных характеристик автомобилей мы будем рассматривать  веса в Lasso-регрессии.

In [459]:
# Посмотрим на отсортированную таблицу с важностью признаков в модели градиентного бустинга
sorted_btg_feature_importances = btg_model_feature_importances.sort_values(by='Feature Importance', ascending=False)
sorted_btg_feature_importances

,Feature Importance
scaling__Valves,0.239467
scaling__Mileage,0.204344
ohe__Brand_Ferrari,0.159720
scaling__Engine Power,0.105526
scaling__Year,0.085576
...,...
ohe__Brand_Scion,0.000000
ohe__Brand_MINI,0.000000
ohe__Brand_Saab,0.000000
ohe__Brand_Plymouth,0.000000


In [460]:
# Рассмотрим первые 10 самых важных признаков
sorted_btg_feature_importances.head(10)

,Feature Importance
scaling__Valves,0.239467
scaling__Mileage,0.204344
ohe__Brand_Ferrari,0.159720
scaling__Engine Power,0.105526
scaling__Year,0.085576
ohe__Origin_British,0.021569
scaling__MPG_mean,0.017586
scaling__Capacity,0.016492
scaling__Usage Intensity,0.016430
scaling__Fuel Efficiency,0.013405


__Вывод:__ Весьма неожиданно, что наиболее важным признаком является количество клапанов. Также есть и предсказуемые результаты, в частности пробег и год выпуска, а также созданный нами признак, характеризующий мощность двигателя.

In [461]:
# Рассмотрим первые 10 самых менее важных признаков
sorted_btg_feature_importances.tail(10)

,Feature Importance
ohe__Exterior color_Gold,0.000001
ohe__Brand_FIAT,0.000000
ohe__Brand_Rolls-Royce,0.000000
ohe__Exterior color_Brown,0.000000
ohe__Brand_Land Rover,0.000000
ohe__Brand_Scion,0.000000
ohe__Brand_MINI,0.000000
ohe__Brand_Saab,0.000000
ohe__Brand_Plymouth,0.000000
ohe__Brand_Pontiac,0.000000


__Вывод:__ Здесь же представлены значения категориальных признаков, встречающихся относительно нечасто, поэтому ничего неожиданного не наблюдается.

---

In [481]:
# Посмотрим на отсортированную таблицу с важностью признаков Lasso-регрессии
sorted_lasso_lr_feature_coefs = lasso_lr_feature_coefs.sort_values(by='Coef', ascending=False)
sorted_lasso_lr_feature_coefs

,Coef
ohe__Brand_Lamborghini,233631.450181
ohe__Brand_Ferrari,225050.065391
ohe__Brand_Rolls-Royce,171100.182785
ohe__Brand_McLaren,102229.201035
ohe__Brand_Aston Martin,99247.626313
...,...
ohe__Fuel type_Gasoline,-11353.316473
ohe__Fuel type_E85 Flex Fuel,-15896.698250
ohe__Configuration_V10,-20509.200869
ohe__Brand_Land Rover,-27783.693849


#### Еще раз перечислим гипотезы:

* *Пробег автомобиля отрицательно влияет на цену*: качество автомобиля с ростом пробега снижается


* *Год выпуска положительно влияет на цену*: при прочих равных чем новее автомобиль, тем он дороже


* *Происхождение автомобиля "British" положительно влияет на цену*: среди британских автомобилей в основном представлены только премиальные бренды, поэтому, на наш взгляд, вес этого признака будет большим


* *Признак "Ownership History" будет значительно влиять на цену: плохая история автомобиля будет отрицательно влиять на цену, а хорошая - положительно

In [496]:
sorted_lasso_lr_feature_coefs.loc[
    [
    'scaling__Mileage', 'scaling__Year', 'ohe__Origin_British',
    'ohe__Ownership History_Good', 'ohe__Ownership History_Potential Risk'
    ]
]

,Coef
scaling__Mileage,-4333.666462
scaling__Year,10798.920597
ohe__Origin_British,36950.211755
ohe__Ownership History_Good,-0.000000
ohe__Ownership History_Potential Risk,1247.861045


__Вывод:__ 

* Неожиданно, пробег автомобиля отрицательно влияет на цену. Данный результат мог, например, появиться вследствие того, что редкие и дорогоие автомобили имели большой пробег


* Ожидаемо, год выпуска положительно влиял на цену автомобиля


* Также подтвердилась гипотеза о положительном влиянии происхождения автомобиля "British"


* Что касается гипотезы относительно признака "Ownership History", то она опровергнута на наших данных, но этот факт можно объяснить тем, что цены на площадке выбирает владелец автомобиля, для которого история владения не так важна